## Estudo de Proteção de Subestação de Medição e Proteção - Parte 1

### Importando Biblioteca (cmath, matplotlib.pyplot e numpy)

In [1]:
run Library.py

### Dados de Entrada fornecidos pela Concessionária
1. Curto circuito
2. Impedância da rede
3. Ajustes do religador

In [2]:
run Dados_Entrada_Concessionária.py

In [3]:
print("O nível de Curto circuito é: ", niveis_curto_circuito)
print("O valor da Impedância da rede é: ", impedancias)
print("O Ajuste do Religador é: ", rele)

O nível de Curto circuito é:  {'Icc3f': 1.619, 'Icc3fassim': 1.684, 'Icc2f': 1.402, 'Iccftmax': 1.169, 'Iccftmin': 183}
O valor da Impedância da rede é:  {'Z1': (1.4507+2.1381j), 'Z0': (1.807+5.3744j)}
O Ajuste do Religador é:  {'tipo': '351-R', 'rtc': 500, 'fase': {'pickup': 230, 'tap_inst': 895, 'curva': 'DEF_05'}, 'neutro': {'pickup': 25, 'tap_inst': 550}, 'religamentos': [5, 20, 40]}


### Cálculos de Potência, Tensão e Corrente de base

In [4]:
tensao_base = 13.80 * 1e3  # em volts
vb = tensao_base
potencia_base = 100e6  # em watts (100 MVA)
sb = potencia_base
zb = vb**2 / sb  # impedância de base
ib = sb / (vb * cmath.sqrt(3))  # corrente de base

print("O valor da Potência de base é: ", sb)
print("O valor do Módulo da Impedância de base é: ", zb)
print("O valor da Corrente de base é: ", ib)

O valor da Potência de base é:  100000000.0
O valor do Módulo da Impedância de base é:  1.9044
O valor da Corrente de base é:  (4183.697602823375+0j)


### Cálculos de Impedância da Rede

In [5]:
z1r = impedancias["Z1"]  # impedância da rede
z0r = impedancias["Z0"]  # impedância da malha de terra
rf = 40  # resistência de falta

print("O valor da Impedância Positiva da rede é em pu: ", z1r)
print("O valor da Impedância Zero da rede é em pu: ", z0r)
print("O valor da Resistência de falta é em ohm: ", rf)

O valor da Impedância Positiva da rede é em pu:  (1.4507+2.1381j)
O valor da Impedância Zero da rede é em pu:  (1.807+5.3744j)
O valor da Resistência de falta é em ohm:  40


### Dados da Subestação do Cliente

In [6]:
run Subestação_Cliente.py

In [7]:
print("Transfomador 1 tem Potência de ",tf1/1000, "kVA e Impedância Perccentual de ",ztf1, "e fator k de ",kinrush1)
print("Transfomador 2 tem Potência de ",tf2/1000, "kVA e Impedância Perccentual de ",ztf2, "e fator k de ",kinrush2)

Transfomador 1 tem Potência de  360.0 kVA e Impedância Perccentual de  0.055 e fator k de  10
Transfomador 2 tem Potência de  225.0 kVA e Impedância Perccentual de  0.045 e fator k de  8


### Cálculos da Impedância do cabo

In [8]:
z_un_cb = complex(r_un_cb,x_un_cb)  # (ohm/km) procurar valor na tabela do fabricante
z1cbpu = z_un_cb * lcb / 1000  # impedância cabo em pu
print("Impedância positiva do cabo em pu", z1cbpu)

Impedância positiva do cabo em pu (1.675+0.3675j)


### Cálculo das correntes dos Transformadores da Subestação de Energia

In [9]:
potencia_instalada = 2e6  # em watts (2 MVA)
intf1 = tf1 / (vb * cmath.sqrt(3))  # corrente no trafo 1
intf2 = tf2 / (vb * cmath.sqrt(3))  # corrente no trafo 2

print("A Corrente Nominal no primário do Trafo 1 é: ",intf1," A")
print("A Corrente Nominal no primário do Trafo 2 é: ",intf2," A")

A Corrente Nominal no primário do Trafo 1 é:  (15.06131137016415+0j)  A
A Corrente Nominal no primário do Trafo 2 é:  (9.413319606352594+0j)  A


### Cálculo das Correntes de curto na barra de media tensão

In [10]:
Z1 = z1r + z1cbpu  # impedância de sequência positiva total
I3ph = ib * 1 / abs(Z1)  # curto trifásico
Fa = cmath.sqrt(1 + 2 * cmath.exp(-2 * cmath.pi * Z1.real / Z1.imag))  # fator de assimetria
I3pha = I3ph * Fa  # corrente de curto assimétrica
I1t = abs(ib * 3 / (2 * Z1 + z0r))  # corrente de curto fase terra
I1ta = I1t * Fa  # corrente de curto fase terra assimétrica
Zf = 3 * rf / zb  # impedância de falta terra
I1tmin = ib * 3 / abs(2 * Z1 + z0r + Zf)  # corrente de falta terra mínima

print("Curto trifásico na barra de média tensão é: ",abs(I3ph), "A")
print("Curto trifásico assimétrico na barra de média tensão é: ",abs(I3pha), "A")
print("Curto fase terra na barra de média tensão é: ",abs(I1t), "A")
print("Curto fase terra assimétrico na barra de média tensão é: ",abs(I1ta), "A")
print("Corrente de falta terra mínimo na barra de média tensão é: ",abs(I1tmin), "A")

Curto trifásico na barra de média tensão é:  1044.3579989447574 A
Curto trifásico assimétrico na barra de média tensão é:  1044.7697967110562 A
Curto fase terra na barra de média tensão é:  954.7981293640297 A
Curto fase terra assimétrico na barra de média tensão é:  955.1746130385318 A
Corrente de falta terra mínimo na barra de média tensão é:  174.74498985584682 A


### Cálculo da Correntes de curto na barra de baixa tensão do Trafo 1

In [11]:
abs_Z1tf1_bt = ztf1 * vb**2 / (tf1 * zb)  # modulo impedância de sequência positiva em bt do trafo 1
Z1tf1_bt = abs_Z1tf1_bt * (0.1 + 0.995j)  # impedância de sequência positiva em bt do trafo 1 em pu
Z0tf1_bt_tf1 = 0.85 * Z1tf1_bt  # impedância de sequência zero em bt do trafo 1 em pu
Z1total_bt_tf1 = z1r + z1cbpu + Z1tf1_bt  # impedância de sequência positiva total em bt do trafo 1
Z0total_bt_tf1 = Z0tf1_bt_tf1  # impedância de sequência zero total em bt do trafo 1
I3ph_bt_tf1 = ib * 1 / abs(Z1total_bt_tf1)  # curto trifásico
Fa_bt_tf1 = cmath.sqrt(1 + 2 * cmath.exp(-2 * cmath.pi * Z1total_bt_tf1.real / Z1total_bt_tf1.imag))  # fator de assimetria
I3pha_bt_tf1 = I3ph_bt_tf1 * Fa_bt_tf1  # corrente de curto assimétrica
I1t_bt_tf1 = abs(ib * 3 / (2 * Z1total_bt_tf1 + Z0total_bt_tf1))  # corrente de curto fase terra
I1ta_bt_tf1 = I1t * Fa_bt_tf1  # corrente de curto fase terra assimétrica
I1tmin_bt_tf1 = ib * 3 / abs(2 * Z1total_bt_tf1 + Z0total_bt_tf1 + Zf)  # impedância de falta terra mínima

print("Curto trifásico na barra de baixa tensão do Trafo 1 é: ",abs(I3ph_bt_tf1), "A")
print("Curto trifásico assimétrico na barra de baixa tensão do Trafo 1 é: ",abs(I3pha_bt_tf1), "A")
print("Curto fase terra na barra de baixa tensão do Trafo 1 é:: ",abs(I1t_bt_tf1), "A")
print("Curto fase terra assimétrico na barra de baixa tensão do Trafo 1 é: ",abs(I1ta_bt_tf1), "A")
print("Corrente de falta terra mínimo na barra de baixa tensão do Trafo 1 é: ",abs(I1tmin_bt_tf1), "A")

Curto trifásico na barra de baixa tensão do Trafo 1 é:  228.51423675409075 A
Curto trifásico assimétrico na barra de baixa tensão do Trafo 1 é:  268.7953084130615 A
Curto fase terra na barra de baixa tensão do Trafo 1 é::  253.6342712015924 A
Curto fase terra assimétrico na barra de baixa tensão do Trafo 1 é:  1123.1040188134984 A
Corrente de falta terra mínimo na barra de baixa tensão do Trafo 1 é:  142.51724301474135 A


### Cálculo da Correntes de curto na barra de baixa tensão do Trafo 2

In [12]:
abs_Z1tf2_bt = ztf2 * vb**2 / (tf2 * zb)  # modulo impedância de sequência positiva em bt do trafo 2
Z1tf2_bt = abs_Z1tf2_bt * (0.1 + 0.995j)  # impedância de sequência positiva em bt do trafo 2 em pu
Z0tf2_bt = 0.85 * Z1tf2_bt  # impedância de sequência zero em bt do trafo 2 em pu
Z1total_bt_tf2 = z1r + z1cbpu + Z1tf2_bt  # impedância de sequência positiva total em bt do trafo 2
Z0total_bt_tf2 = Z0tf2_bt  # impedância de sequência zero total em bt do trafo 2
I3ph_bt_tf2 = ib * 1 / abs(Z1total_bt_tf2)  # curto trifásico
Fa_bt_tf2 = cmath.sqrt(1 + 2 * cmath.exp(-2 * cmath.pi * Z1total_bt_tf2.real / Z1total_bt_tf2.imag))  # fator de assimetria
I3pha_bt_tf2 = I3ph_bt_tf2 * Fa_bt_tf2  # corrente de curto assimétrica
I1t_bt_tf2 = abs(ib * 3 / (2 * Z1total_bt_tf2 + Z0total_bt_tf2))  # corrente de curto fase terra
I1ta_bt_tf2 = I1t * Fa_bt_tf2  # corrente de curto fase terra assimétrica
I1tmin_bt_tf2 = ib * 3 / abs(2 * Z1total_bt_tf2 + Z0total_bt_tf2 + Zf)  # impedância de falta terra mínima

print("Curto trifásico na barra de baixa tensão do Trafo 2 é: ",abs(I3ph_bt_tf2), "A")
print("Curto trifásico assimétrico na barra de baixa tensão do Trafo 2 é: ",abs(I3pha_bt_tf2), "A")
print("Curto fase terra na barra de baixa tensão do Trafo 2 é:: ",abs(I1t_bt_tf2), "A")
print("Curto fase terra assimétrico na barra de baixa tensão do Trafo 2 é: ",abs(I1ta_bt_tf2), "A")
print("Corrente de falta terra mínimo na barra de baixa tensão do Trafo 2 é: ",abs(I1tmin_bt_tf2), "A")

Curto trifásico na barra de baixa tensão do Trafo 2 é:  182.02317101659045 A
Curto trifásico assimétrico na barra de baixa tensão do Trafo 2 é:  221.0731907544342 A
Curto fase terra na barra de baixa tensão do Trafo 2 é::  199.6274898953788 A
Curto fase terra assimétrico na barra de baixa tensão do Trafo 2 é:  1159.6340609055328 A
Corrente de falta terra mínimo na barra de baixa tensão do Trafo 2 é:  129.2510836663407 A


### Dimensionamento do TC

In [13]:
Dm = 1400e3  # demanda contratada em W
In = Dm / (vb * 0.92 * cmath.sqrt(3))  # corrente nominal demanda contratada
It = ((tf1 + tf2) / (cmath.sqrt(3) * vb))  # corrente nominal dos transformadores
Intc_int1 = I3ph / 20  # Corrente de primeiro critério de curto no TC
Intc_int2 = 2 * In  # Corrente de segundo critério de curto no TC
Intc_int3 = 2 * It  # Corrente de terceiro critério de curto no TC

print("Corrente de primeiro critério de curto no TC é: ",abs(Intc_int1), "A")
print("Corrente de segundo critério de curto no TC é: ",abs(Intc_int2), "A")
print("Corrente de teceiro critério de curto no TC é: ",abs(Intc_int3), "A")

if abs(Intc_int1) > 100 or abs(Intc_int2) > 100 or abs(Intc_int3) > 100:
    Intc = 150
else:
    Intc = 100

print("TC escolhiddo é de ",Intc,"/5  A")

Corrente de primeiro critério de curto no TC é:  52.21789994723787 A
Corrente de segundo critério de curto no TC é:  127.32992704245056 A
Corrente de teceiro critério de curto no TC é:  48.94926195303349 A
TC escolhiddo é de  150 /5  A


### Primeiro Ajuste de Proteção do Relé

In [14]:
# --------- corrente de magnetização dos transformadores ----------- #
Imag = intf2 * kinrush2 + intf1  # corrente magnetização dos transformadores

# --------- ponto ANSI dos transformadores ----------- #
IANSI_tf1 = intf1 / ztf1  # corrente magnetização dos transformador 1
TANSI_tf1 = ztf1**2 / 8  # tempo de disparo ANSI transformador 1
IANSI_tf2 = intf2 / ztf2  # corrente magnetização dos transformador 2
TANSI_tf2 = ztf2**2 / 8  # tempo de disparo ANSI transformador 2

# --------- ajustes de curvas de proteção ----------- #
Ipf_inst = Dm / (vb * cmath.sqrt(3))  # corrente de partida da fase demanda
Iif = 1.4 * Imag  # corrente instatânea
Ipn = 0.1 * Ipf_inst  # corrente de partida de neutro
Iin = 0.2 * Iif  # corrente instatânea de fase

print("Corrente pickup de fase é: ",abs(Ipf_inst),"A")
print("Corrente instatânea de fase é: ",abs(Iif),"A")
print("Corrente pickup de neutro é: ",abs(Ipn),"A")
print("Corrente instatânea de neutro é: ",abs(Iin),"A")

Corrente pickup de fase é:  58.57176643952725 A
Corrente instatânea de fase é:  126.51501550937886 A
Corrente pickup de neutro é:  5.857176643952726 A
Corrente instatânea de neutro é:  25.303003101875774 A
